こういう感じのコメントを付ける

```
constexpr double LINE_LENGTH_LOSS_STEPPING_PROGRESS_RATE = 0.7001976706415303;  // OPTIMIZE [0.0, 1.0]
constexpr double  LINE_LENGTH_LOSS_WEIGHT = 8.009299591633644e-06;  // OPTIMIZE [1e-8, 1e-4] LOG
constexpr int N_RANDOM_FACE_CHOICE = 3;  // OPTIMIZE [1, 10]
std::string hoge="hoge";//OPTIMIZE{"11", "unnn" ,"ooo"}
```

`[` と `,` の間の文字列が Python の式として評価され、その結果の型が最適化に使われる

/* でのコメントアウトは効かないので注意

In [1]:
import re
import sys
from time import sleep
from pprint import pprint
from functools import partial

import optuna
try:
    import pandas as pd
except:
    pass


FILENAME = "answer.cpp"

In [2]:
!mkdir optimize_tmp

In [4]:
# 元のファイルのコンパイル
!g++ {FILENAME} -std=gnu++17 -Wall -Wextra -O2 -DONLINE_JUDGE -o {FILENAME}.out

answer.cpp: In constructor ‘State::State(const Input&)’:
answer.cpp:393:30: warning: ‘State::dfs_path’ will be initialized after [-Wreorder]
  393 |  Stack<Direction, MAX_DEPTH> dfs_path;
      |                              ^~~~~~~~
answer.cpp:391:22: warning:   ‘Stack<int, 2500> State::closed’ [-Wreorder]
  391 |  Stack<int, 50 * 50> closed;
      |                      ^~~~~~
answer.cpp:401:2: warning:   when initialized here [-Wreorder]
  401 |  State(const Input& a_input) : input(a_input), path(), dfs_path(), closed(),
      |  ^~~~~


In [5]:
# テストケース生成
!wget https://img.atcoder.jp/ahc002/c993bb7f09d9f8857fc90951fc6af11d.zip
!unzip c993bb7f09d9f8857fc90951fc6af11d.zip

SEED = 201
N_TESTCASES = 50
!seq {SEED} {SEED+N_TESTCASES-1} | cargo run --release --manifest-path ./tools/Cargo.toml --bin gen

--2021-04-25 13:07:58--  https://img.atcoder.jp/ahc002/c993bb7f09d9f8857fc90951fc6af11d.zip
Resolving img.atcoder.jp (img.atcoder.jp)... 13.249.87.24, 13.249.87.87, 13.249.87.58, ...
Connecting to img.atcoder.jp (img.atcoder.jp)|13.249.87.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31143 (30K) [application/zip]
Saving to: ‘c993bb7f09d9f8857fc90951fc6af11d.zip’

c993bb7f09d9f8857fc 100%[===================>]  30.41K  --.-KB/s    in 0.1s    

2021-04-25 13:07:59 (243 KB/s) - ‘c993bb7f09d9f8857fc90951fc6af11d.zip’ saved [31143/31143]

Archive:  c993bb7f09d9f8857fc90951fc6af11d.zip
   creating: tools/
  inflating: tools/Cargo.toml        
  inflating: tools/README.html       
  inflating: tools/README.md         
  inflating: tools/seeds.txt         
  inflating: tools/vis.html          
  inflating: tools/Cargo.lock        
  inflating: tools/out.svg           
  inflating: tools/out.txt           
   creating: tools/src/
  inflating: tools/src/lib.rs     

In [8]:
!tools/target/release/gen tools/seeds.txt

In [10]:
!ls

answer.cpp	c993bb7f09d9f8857fc90951fc6af11d.zip  optimize_tmp     tools
answer.cpp.out	in				      optimizer.ipynb


In [11]:
# ここでエラー出力は何かおかしいかもしれない


# パラメータ抽出
with open(FILENAME) as f:
    answer = f.read()


variables_optimize = []

for left, value, right, options in re.findall(r"^([^/\n]*=\s*)(.+?)(\s*;\s*//(?:.*\W)?OPTIMIZE(\W.*))$", answer, re.MULTILINE):
    name = left.replace("=", "").strip().split()[-1]
    
    searched = re.search(r".*\[(?P<min>.*),(?P<max>.*)\].*", options)
    if searched:
        min_value = max_value = None
        try:
            min_value = eval(searched.group("min"))
            max_value = eval(searched.group("max"))
            assert min_value <= max_value
        except Exception as e:
            print(f"searched={searched}", file=sys.stderr)
            print(e, file=sys.stderr)
            continue
        log = "LOG" in options  # 雑、直したほうが良い
        if type(min_value) != type(max_value):
            print(f"searched={searched}", file=sys.stderr)
            print("types not matched", file=sys.stderr)
            continue
        if isinstance(min_value, int):
            method = "suggest_int"
        elif isinstance(min_value, float):
            method = "suggest_float"
        else:
            print(f"searched={searched}", file=sys.stderr)
            print(f"unknown type ({type(min_value)})", file=sys.stderr)
            continue
        variables_optimize.append({
            "name": name,
            "method": method,
            "min": min_value,
            "max": max_value,
            "log": log,
            "left": left,
            "right": right,
        })
    elif searched := re.search(r".*\{(?P<choices>.*?)\}.*", options):
        choices = list(map(lambda x: x.strip(), searched.group("choices").split(",")))
        variables_optimize.append({
            "name": name,
            "method": "suggest_categorical",
            "choices": choices,
            "left": left,
            "right": right,
        })
    else:
        print(f"searched={searched}", file=sys.stderr)
        print(f"pattern was matched but options are incorrect.", file=sys.stderr)

print(len(variables_optimize), "variables were found.")
if globals().get("pd"):
    display(pd.DataFrame(variables_optimize))
else:
    pprint(variables_optimize)

4 variables were found.


,name,method,min,max,log,left,right
0,COEF_CENTER_L1,suggest_float,-100.0,100.0,False,\t\tconstexpr double COEF_CENTER_L1 =,"; // OPTIMIZE [-100.0, 100.0]"
1,COEF_CENTER_L1_SQ,suggest_float,0.0,100.0,False,\t\tconstexpr double COEF_CENTER_L1_SQ =,"; // OPTIMIZE [0.0, 100.0]"
2,COEF_POINT,suggest_float,0.0,100.0,False,\t\tconstexpr double COEF_POINT =,"; // OPTIMIZE [0.0, 100.0]"
3,COEF_DIST,suggest_float,0.0,10000.0,False,\t\tconstexpr double COEF_DIST =,"; // OPTIMIZE [0.0, 10000.0]"


In [18]:
def escape(string):  # 正規表現の中でそのまま使いたい文字列をエスケープ
    res = !echo '{string}' | sed -e 's/[]\/$*.^[]/\\&/g'
    return res[0]

def escape_sed(string):  # sed の置換後の文字列用のエスケープ
    res = !echo '{string}' | sed -e 's/[\/&]/\\&/g'
    return res[0]


def objective_with_index(trial, n_internal_parallel):
    index_parallel = trial.number % 100
    print(f"{index_parallel=}")
    
    parameters_changed_filename = f"{index_parallel}_{FILENAME}"
    directory_input = "./in"  # "/" をつけない
    
    !mkdir {index_parallel}_out
    !mkdir {index_parallel}_score
    !rm {index_parallel}_out/*
    !rm {index_parallel}_score/*
    
    # ファイル作成
    !cp {FILENAME} {parameters_changed_filename}
    sed_options = [f"-i {parameters_changed_filename}"]
    for variable in variables_optimize:
        if variable["method"] == "suggest_categorical":
            val =  trial.suggest_categorical(variable["name"], variable["choices"])
        else:
            val = getattr(trial, variable["method"])(variable["name"], variable["min"], variable["max"])
        left = variable["left"]
        right = variable["right"]
        sed_options.append(f"""-e 's/^{escape(left)}.*{escape(right)}$/{escape_sed(left)}{val}{escape_sed(right)}/'""")
    command_sed = f"sed {' '.join(sed_options)}"
    print(command_sed)
    !{command_sed}
    
    # コンパイル
    command_compile = f"g++ {parameters_changed_filename} -std=gnu++17 -O2 -DONLINE_JUDGE -o {parameters_changed_filename}.out"
    print(command_compile)
    !{command_compile}
    # 実行・採点コマンド (@ はファイル名)
    command_exec = (
        f"./{parameters_changed_filename}.out < {directory_input}/@ > ./{index_parallel}_out/@;"
        f"cargo run --release --manifest-path ./tools/Cargo.toml --bin vis {directory_input}/@ ./{index_parallel}_out/@ 2> /dev/null | sed -e s/[^0-9]//g > ./{index_parallel}_score/@;"
    )
    # 並列実行 (sed はパスのディレクトリ部分を消してファイル名にしてる)
    !find {directory_input}/* | sed 's!^.*/!!' | xargs -I@ -P {n_internal_parallel} sh -c '{command_exec}'
    
    # 集計
    file_paths = !find ./{index_parallel}_score/*
    sum_score = 0
    for file_path in file_paths:  
        with open(file_path) as f:
            sum_score += int(f.read())
    return sum_score



In [20]:
storage_path = f"study.db"
storage = f"sqlite:///{storage_path}"
study_name = "study"
study = optuna.create_study(storage=storage, load_if_exists=True, study_name=study_name, direction="maximize")

def callback(study, trial):
    try:
        if study.best_value == trial.value:  ############# 要変更
            print(f"Updated! {study.best_value} -> {trial.value}")
            index_parallel = trial.number % 100
            parameters_changed_filename = f"{index_parallel}_{FILENAME}"
            !cp {parameters_changed_filename} 00000optimized.cpp
        print("callback ok")
    except:
        print(":(")

objective = partial(objective_with_index, n_internal_parallel=4)
study.optimize(objective, n_trials=3000, timeout=3600, n_jobs=6, callbacks=[callback])

[I 2021-04-25 13:16:33,136] Using an existing study with name 'study' instead of creating a new one.


index_parallel=12
index_parallel=13
index_parallel=10
index_parallel=14
index_parallel=11
index_parallel=15
rm: cannot remove '13_out/*': No such file or directory
rm: cannot remove '10_out/*': No such file or directory
rm: cannot remove '12_out/*': No such file or directory
rm: cannot remove '11_out/*': No such file or directory
rm: cannot remove '14_out/*': No such file or directory
rm: cannot remove '15_out/*': No such file or directory
rm: cannot remove '12_score/*': No such file or directory
rm: cannot remove '10_score/*': No such file or directory
rm: cannot remove '13_score/*': No such file or directory
rm: cannot remove '14_score/*': No such file or directory
rm: cannot remove '11_score/*': No such file or directory
rm: cannot remove '15_score/*': No such file or directory
sed -i 11_answer.cpp -e 's/^		constexpr double COEF_CENTER_L1 = .*;  \/\/ OPTIMIZE \[-100\.0, 100\.0\]$/		constexpr double COEF_CENTER_L1 = -31.947897903881596;  \/\/ OPTIMIZE [-100.0, 100.0]/' -e 's/^		const

[I 2021-04-25 13:16:43,400] Trial 10 finished with value: 880778.0 and parameters: {'COEF_CENTER_L1': -39.50803360992359, 'COEF_CENTER_L1_SQ': 29.30153106434714, 'COEF_POINT': 57.9903340573231, 'COEF_DIST': 2648.630075673831}. Best is trial 7 with value: 1255171.0.


callback ok
index_parallel=16


[I 2021-04-25 13:16:43,492] Trial 15 finished with value: 849604.0 and parameters: {'COEF_CENTER_L1': -18.154110756829382, 'COEF_CENTER_L1_SQ': 99.93892995152294, 'COEF_POINT': 31.378239891181426, 'COEF_DIST': 8750.189276372976}. Best is trial 7 with value: 1255171.0.


...

[I 2021-04-25 13:33:27,448] Trial 501 finished with value: 1158894.0 and parameters: {'COEF_CENTER_L1': -66.49421711158415, 'COEF_CENTER_L1_SQ': 12.16581019758797, 'COEF_POINT': 31.03236352050317, 'COEF_DIST': 4772.497751167573}. Best is trial 142 with value: 1556780.0.


callback ok
index_parallel=7
mkdir: cannot create directory ‘7_out’: File exists
mkdir: cannot create directory ‘7_score’: File exists


[I 2021-04-25 13:33:28,105] Trial 502 finished with value: 1421322.0 and parameters: {'COEF_CENTER_L1': 82.44085263659485, 'COEF_CENTER_L1_SQ': 2.7966355385750035, 'COEF_POINT': 6.793303258462788, 'COEF_DIST': 6576.174777826446}. Best is trial 142 with value: 1556780.0.


callback ok
index_parallel=8
mkdir: cannot create directory ‘8_out’: File exists
mkdir: cannot create directory ‘8_score’: File exists


KeyboardInterrupt: 

In [ ]:
# $ apt install sysstat
# $ mpstat -P ALL 5
# とかやると CPU 負荷を見られる

In [19]:
print(f"{study.best_value}")
pprint(study.best_params)

48621734068.0
{'ANNEALING_A': -9.663753766636084,
 'ANNEALING_B': 0.23708439451937813,
 'ANNEALING_END': 0.004139552269746777,
 'ANNEALING_START': 0.4830473145504588,
 'GRADIENT_CLIPPING': 0.00019811172822438805,
 'GRADIENT_DESCENT_LR': 10380014.948175553,
 'GRADIENT_DESCENT_MOMENTUM': 0.3276696959783839,
 'GRADIENT_DESCENT_STEPS': 30,
 'GRADIENT_DESCENT_STEPS_STEPPING_PROGRESS_RATE': 0.9031609842262478,
 'LINE_LENGTH_LOSS_STEPPING_PROGRESS_RATE': 0.021698592868210798,
 'LINE_LENGTH_LOSS_WEIGHT': 9.614201554394316e-07,
 'N_RANDOM_FACE_CHOICE': 8,
 'hoge': '"ooo"'}
